In [1]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install tqdm

You should consider upgrading via the 'C:\Users\farja\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\farja\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\farja\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\farja\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda"
model_id = "gpt2-large"
tokenizer = AutoTokenizer.from_pretrained("../models/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("../models/DialoGPT-medium")

C:\Users\farja\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from datasets import load_dataset

test = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
encodings = tokenizer("\n\n".join(test["text"]), return_tensors="pt")

Dataset wikitext downloaded and prepared to C:\Users\farja\.cache\huggingface\datasets\wikitext\wikitext-2-raw-v1\1.0.0\a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.


Token indices sequence length is longer than the specified maximum sequence length for this model (287644 > 1024). Running this sequence through the model will result in indexing errors


In [6]:

import torch
from tqdm import tqdm

device = 'cpu'
max_length = model.config.n_positions
stride = 512

nlls = []
for i in tqdm(range(0, encodings.input_ids.size(1), stride)):
    begin_loc = max(i + stride - max_length, 0)
    end_loc = min(i + stride, encodings.input_ids.size(1))
    trg_len = end_loc - i  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone().to(device)
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)
        neg_log_likelihood = outputs[0] * trg_len

    nlls.append(neg_log_likelihood)

ppl = torch.exp(torch.stack(nlls).sum() / end_loc)
print(ppl)

  1%|▋                                                                               | 5/562 [01:18<2:25:18, 15.65s/it]


KeyboardInterrupt: 